In [100]:
# Python 3 program for Elo Rating 
import math
import pandas as pd
import numpy as np

# Function to calculate the Probability 
def Probability(rating1, rating2): 
    return 1.0 * 1.0 / (1 + 1.0 * math.pow(10, 1.0 * (rating1 - rating2) / 400)) 

# Function to calculate Elo rating 
# K is a constant. 
# d determines whether Team A wins or Team B. 
def EloRating(Ra, Rb, K, d): 
    # Winning Probability of Team B 
    Pb = Probability(Ra, Rb) 

    #Winning Probability of Team A 
    Pa = Probability(Rb, Ra) 

    # Case -1 When Player A wins 
    if (d == 1) : 
        Ra = Ra + K * (1 - Pa) 
        Rb = Rb + K * (0 - Pb) 
    # Case -2 When Player B wins 
    else : 
        Ra = Ra + K * (0 - Pa) 
        Rb = Rb + K * (1 - Pb) 

    return round(Ra, 6) 

def get_prev_ELO(df, team, date):
    """
    Given a df, team and date, return the team's previous ELO score
    """
    try:
        # all games for this team
        team_df = df[df['Team'] == team]
        # only games before the date specified
        subteam_df = team_df[team_df['Date'] < ck_date]
        # make sure the df is sorted and grab the last row
        subteam_df = subteam_df.sort_values(by=['Date'])
        last_game = subteam_df.tail(1)
        # grab the team's ELO score from that row
        prev_ELO = last_game.iloc[0]['Team_ELO']
    except IndexError:
        prev_ELO = 0
    print(str(date), team + ' prev elo was ' + str(prev_ELO))
    
    return prev_ELO

def calc_team_ELO(df, team, opp, date, d):
    print(date, team)
    team_prev_ELO = get_prev_ELO(df, team, date)
    opp_prev_ELO = get_prev_ELO(df, opp, date)
    
    return EloRating(team_prev_ELO, opp_prev_ELO, 32, d)

def calc_opp_ELO(df, team, opp, date, d):
    print(date, opp)
    team_prev_ELO = get_prev_ELO(df, team, date)
    opp_prev_ELO = get_prev_ELO(df, opp, date)
    
    return EloRating(opp_prev_ELO, team_prev_ELO, 32, d)


In [10]:
past_games = pd.read_csv('./SRinput/current/past_games.csv')
cols = past_games.columns[0:6]
past_games = past_games[cols]
past_games['Date'] = pd.to_datetime(past_games['Date'])
past_games['Team_ELO'] = 0
past_games['Opp_ELO'] = 0

In [11]:
past_games.head()

,Year,Season,Date,Team,Opp,Won,Team_ELO,Opp_ELO
0,2014,2014,2014-08-27 19:00:00,Georgia State,Abilene Christian,1,0,0
1,2014,2014,2014-08-27 19:00:00,Abilene Christian,Georgia State,0,0,0
2,2014,2014,2014-08-28 19:00:00,Akron,Howard,1,0,0
3,2014,2014,2014-08-28 19:00:00,Howard,Akron,0,0,0
4,2014,2014,2014-08-28 22:30:00,Arizona State,Weber State,1,0,0


In [52]:
for index, row in past_games.iterrows():
    past_games.loc[index, 'Team_ELO'] = calc_team_ELO(past_games, row['Team'], row['Opp'], row['Date'], row['Won'])
    past_games.loc[index, 'Opp_ELO'] = calc_opp_ELO(past_games, row['Team'], row['Opp'], row['Date'], not row['Won'])

In [30]:
ck_date = pd.Timestamp(2014, 11, 12)
calc_opp_ELO(past_games, 'Florida', 'Auburn', ck_date, 0)


-16.0

In [58]:
games = past_games.head().copy()

In [60]:
d1 = pd.Timestamp(2014, 1, 1)
d2 = pd.Timestamp(2014, 2, 1)
d3 = pd.Timestamp(2014, 3, 1)
d4 = pd.Timestamp(2014, 4, 1)
d5 = pd.Timestamp(2014, 5, 1)

In [65]:
games['O'] = ['Georgia', 'Akron', 'Akron', 'Florida', 'Georgia']

In [71]:
games

,Year,Season,Date,Team,Opp,Won,Team_ELO,Opp_ELO,D,T,O
0,2014,2014,2014-08-27 19:00:00,Georgia State,Abilene Christian,1,16.0,-16.0,2014-01-01,Florida,Georgia
1,2014,2014,2014-08-27 19:00:00,Abilene Christian,Georgia State,0,-16.0,16.0,2014-02-01,Georgia,Akron
2,2014,2014,2014-08-28 19:00:00,Akron,Howard,1,16.0,-16.0,2014-03-01,Florida,Akron
3,2014,2014,2014-08-28 19:00:00,Howard,Akron,0,-16.0,16.0,2014-04-01,Georgia,Florida
4,2014,2014,2014-08-28 22:30:00,Arizona State,Weber State,1,16.0,-16.0,2014-05-01,Florida,Georgia


In [77]:
cols = ['Year', 'Season', 'D', 'T', 'O', 'Won']
gs = games[cols]
gs['Team_ELO'] = 0
gs['Opp_ELO'] = 0

/anaconda3/envs/TensorFlow/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/anaconda3/envs/TensorFlow/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [78]:
gs.columns = ['Year', 'Season', 'Date', 'Team', 'Opp', 'Won', 'Team_ELO', 'Opp_ELO']
gs

,Year,Season,Date,Team,Opp,Won,Team_ELO,Opp_ELO
0,2014,2014,2014-01-01,Florida,Georgia,1,0,0
1,2014,2014,2014-02-01,Georgia,Akron,0,0,0
2,2014,2014,2014-03-01,Florida,Akron,1,0,0
3,2014,2014,2014-04-01,Georgia,Florida,0,0,0
4,2014,2014,2014-05-01,Florida,Georgia,1,0,0


In [83]:
gs['Won'] = [1, 0, 1, 0, 0]
gs

/anaconda3/envs/TensorFlow/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,Year,Season,Date,Team,Opp,Won,Team_ELO,Opp_ELO
0,2014,2014,2014-01-01,Florida,Georgia,1,0,0
1,2014,2014,2014-01-01,Georgia,Florida,0,0,0
2,2014,2014,2014-02-01,Florida,Georgia,1,0,0
3,2014,2014,2014-02-01,Georgia,Florida,0,0,0
4,2014,2014,2014-03-01,Florida,Georgia,0,0,0


In [104]:
for index, row in gs.iterrows():
    print('****GAME****')
    gs.loc[index, 'Opp_ELO'] = calc_opp_ELO(gs, row['Team'], row['Opp'], row['Date'], not row['Won'])
    gs.loc[index, 'Team_ELO'] = calc_team_ELO(gs, row['Team'], row['Opp'], row['Date'], row['Won'])
gs

****GAME****
2014-01-01 00:00:00 Georgia
2014-01-01 00:00:00 Florida prev elo was 0
2014-01-01 00:00:00 Georgia prev elo was 0
2014-01-01 00:00:00 Florida
2014-01-01 00:00:00 Florida prev elo was 0
2014-01-01 00:00:00 Georgia prev elo was 0
****GAME****
2014-01-01 00:00:00 Florida
2014-01-01 00:00:00 Georgia prev elo was 0.0
2014-01-01 00:00:00 Florida prev elo was 16.0
2014-01-01 00:00:00 Georgia
2014-01-01 00:00:00 Georgia prev elo was 0.0
2014-01-01 00:00:00 Florida prev elo was 16.0
****GAME****
2014-02-01 00:00:00 Georgia
2014-02-01 00:00:00 Florida prev elo was 16.0
2014-02-01 00:00:00 Georgia prev elo was -15.263693
2014-02-01 00:00:00 Florida
2014-02-01 00:00:00 Florida prev elo was 16.0
2014-02-01 00:00:00 Georgia prev elo was -15.263693
****GAME****
2014-02-01 00:00:00 Florida
2014-02-01 00:00:00 Georgia prev elo was -15.263693
2014-02-01 00:00:00 Florida prev elo was 16.0
2014-02-01 00:00:00 Georgia
2014-02-01 00:00:00 Georgia prev elo was -15.263693
2014-02-01 00:00:00 Flor

,Year,Season,Date,Team,Opp,Won,Team_ELO,Opp_ELO
0,2014,2014,2014-01-01,Florida,Georgia,1,16.000000,-16.000000
1,2014,2014,2014-01-01,Georgia,Florida,0,-15.263693,31.263693
2,2014,2014,2014-02-01,Florida,Georgia,1,30.564127,-29.827820
3,2014,2014,2014-02-01,Georgia,Florida,0,-29.827820,30.564127
4,2014,2014,2014-03-01,Florida,Georgia,0,-1.435873,2.172180


In [103]:
gs['Team_ELO'] = 0
gs['Opp_ELO'] = 0

/anaconda3/envs/TensorFlow/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/anaconda3/envs/TensorFlow/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [95]:
EloRating(16, -16, 32, 1)

30.530498

In [98]:
ck_date = pd.Timestamp(2014, 2, 1)
calc_team_ELO(gs, 'Florida', 'Georgia', ck_date, 1)

2014-02-01 00:00:00 Florida
2014-02-01 00:00:00 Florida prev elo was 0
2014-02-01 00:00:00 Georgia prev elo was 0


16.0